# Filter Bubble | Visualisations

This _Jupyter Notebook_ generates visualisations based on mobile browser history and a survey.

## Import

Let's start by importing the relevant packages.

In [ ]:
import pandas

## Qualtrics

Both the _Qualtrics_ survey data and the _Passive Data Kit_ browser history data are privacy sensitive, so the _data_ directory is _gitignored_ in the GitHub repository. Also remember to _clear_ all output cells before saving this _Jupyter Notebook_ and pushing it to the repository, to ensure no personal information is released.

### Data File

The Qualtrics data was exported in _CSV_ format and saved in the _data_ directory.

In [ ]:
dataFileQualtrics = "data\Qualtrics.csv"

We can import this into a _Pandas dataframe_, taking into account the relevant _dtypes_.

In [ ]:
qualtricsUnionWaves = pandas.read_csv(dataFileQualtrics, sep=",", header=[0,1,2], dtype={'Duration (in seconds)': int})
qualtricsUnionWaves.head(3)

The data file contains three rows as headers, however, we would like to only keep one. First, the second and third row are saved in a seperate dataframe.

In [ ]:
qualtricsHeader = qualtricsUnionWaves.columns.to_frame().T.drop(0)
qualtricsHeader.columns = qualtricsHeader.columns.droplevel([1, 2])
qualtricsHeader

Then, these rows are removed from the Qualtrics dataframe.

In [ ]:
qualtricsUnionWaves.columns = qualtricsUnionWaves.columns.droplevel([1, 2])
qualtricsUnionWaves.head(3)

The column _Q70_ contains the email address of the participant. This is needed to map the entries of the first wave and the second wave of Qualtrics data.

In [ ]:
qualtricsHeader[["Q70"]]

Let's hash this column so we can visualise the data in this _Jupyter Notebook_ without immediately exposing privacy sensitive data.

In [ ]:
qualtricsUnionWaves["Q70"] = [hash(x) for x in qualtricsUnionWaves["Q70"]]

We will set column _Q70_ as the index of the dataframe.

In [ ]:
qualtricsUnionWaves = qualtricsUnionWaves.set_index("Q70")
qualtricsUnionWaves.head(3)

Then, we split the data into first and second wave using the answer to question _Q72_.

In [ ]:
qualtricsHeader[["Q72"]]

We'll also append numbers '_1' and '_2' to the column names to differentiate between the two waves.

In [ ]:
qualtricsFirstWave = qualtricsUnionWaves[qualtricsUnionWaves["Q72"] == "Ja"]
qualtricsFirstWave.columns = [x + "_1" for x in qualtricsFirstWave.columns]
qualtricsFirstWave.head(3)

In [ ]:
qualtricsSecondWave = qualtricsUnionWaves[qualtricsUnionWaves["Q72"] == "Nee"]
qualtricsSecondWave.columns = [x + "_2" for x in qualtricsSecondWave.columns]
qualtricsSecondWave.head(3)

Let's start by listing hashed email addresses of all participants who showed up in the first wave.

In [ ]:
participantsFirstWave = qualtricsFirstWave.index.unique()
len(participantsFirstWave)

Some participants may drop out between the first and second wave. These will only show up in the index of the dataframe of the first wave, but not in the index of the dataframe of the second wave.

In [ ]:
participantsSecondWave = qualtricsSecondWave.index.unique()
len(participantsSecondWave)

We can see which participants entered both waves by examining the intersection of the indices of both dataframes.

In [ ]:
participantsIntersectionWaves = qualtricsSecondWave.index.intersection(participantsFirstWave)
len(participantsIntersectionWaves)

Some _only_ did the first wave, which we can find by taking the difference with the index of the second wave dataframe.

In [ ]:
participantsOnlyFirstWave = participantsFirstWave.difference(participantsSecondWave)
len(participantsOnlyFirstWave)

It may occur that some entered _Yes_ to question _Q72_ even though they did not participate in both waves. This can be examined by taking the differences of the indices.

In [ ]:
participantsOnlySecondWave = qualtricsSecondWave.index.difference(participantsFirstWave)
len(participantsOnlySecondWave)

Such participants may simply be removed from the data, but let's examine these outliers more carefully first. We can list the number of unanswered questions and compare this with the number of unanswered questions of other participants.

In [ ]:
qualtricsSecondWave.loc[participantsOnlySecondWave].T.isna().sum().to_frame()

In [ ]:
qualtricsSecondWave.loc[participantsIntersectionWaves].T.isna().sum().mean()

We can list the time they spent on the survey and compare this to the average time spent by other participants.

In [ ]:
qualtricsSecondWave.loc[participantsOnlySecondWave][["Duration (in seconds)_2"]]

We can list the time they spent on the survey and compare this to the average time spent by other participants.

In [ ]:
qualtricsSecondWave.loc[participantsIntersectionWaves]["Duration (in seconds)_2"].mean()

If we determine these participants did not take the survey seriously, we can remove them from the data and continue with a smaller dataframe for the second wave.

In [ ]:
qualtricsSecondWave = qualtricsSecondWave.loc[participantsIntersectionWaves]

We can join the dataframes of the first and of the second wave on the indices (the hashed email addresses), remembering that we had already appended '_1' and '_2' to the column names previously to differentiate between the waves.

In [ ]:
dataFrameQualtrics = pandas.concat([qualtricsFirstWave, qualtricsSecondWave], axis=1, sort=False)
dataFrameQualtrics.head(3)

Each participants was also given an ID by _Web Historian_, though a different one in the first and in the second wave. Now that the dataframes are joined, we can see which ID's belong together. Only participants who finished the second wave will have a second ID.

In [ ]:
qualtricsIdMap = dataFrameQualtrics[["idRcvd_1", "idRcvd_2"]]
qualtricsIdMap.head(5)

Let's also merge these Web Historians ID's into a list.

In [ ]:
qualtricsIdListFirstWave = dataFrameQualtrics["idRcvd_1"][pandas.notna(dataFrameQualtrics["idRcvd_1"])].tolist()
qualtricsIdListSecondWave = dataFrameQualtrics["idRcvd_2"][pandas.notna(dataFrameQualtrics["idRcvd_2"])].tolist()
qualtricsIdListUnionWaves = qualtricsIdListFirstWave + qualtricsIdListSecondWave
qualtricsIdListUnionWaves

For participants who entered both waves, this can be transformed into a dictionary from the first wave ID to the second wave ID.

In [ ]:
qualtricsIdDictionary = qualtricsIdMap.loc[participantsIntersectionWaves].set_index("idRcvd_1")["idRcvd_2"].to_dict()
qualtricsIdDictionary

### Simple Metrics

Let's look at some descriptive statistics of the data given by participants of both waves. We can start by examining the number of missing values per question.

In [ ]:
countNansQualtricsIntersectionWaves = dataFrameQualtrics.loc[participantsIntersectionWaves].isna().sum().to_frame()
countNansQualtricsIntersectionWaves.T

We can also plot this, remembering that when we joined the first and second wave data, every question was duplicated. Therefore, it makes sense that we should find large numbers of missing values.

In [ ]:
ax = countNansQualtricsIntersectionWaves.plot(kind="bar", figsize=(15,5), legend=False)
ax.set(ylabel="NaNs", title="Number of NaN responses per question")
ax

Perhaps more interestingly, let's examine the number of missing values per person.

In [ ]:
countNansPerPersonQualtricsIntersectionWaves = dataFrameQualtrics.loc[participantsIntersectionWaves].T.isna().sum().to_frame()
countNansPerPersonQualtricsIntersectionWaves.T

This, again, can be plotted.

In [ ]:
ax = countNansPerPersonQualtricsIntersectionWaves.plot(kind="bar", figsize=(15,5), legend=False)
ax.set(ylabel="NaNs", title="Number of NaN responses per person")
ax

Let's also examine the time is took for each participant to finish the survey.

In [ ]:
durationPerPersonQualtrics = dataFrameQualtrics[["Duration (in seconds)_1"]]
durationPerPersonQualtrics.T

This, again, can be plotted.

In [ ]:
ax = durationPerPersonQualtrics.plot(kind="bar", figsize=(15,5), legend=False)
ax.set(ylabel="Time in seconds", title="Duration per person")
ax

## Passive Data Kit

### Data File

The _Passive Data Kit_ data is automaticaly split into multiple parts by the server when it is exported. Let's import each of these files into the same dataframe.

In [ ]:
dataFilesPassiveDataKit = ["data\pdk_web-historian_1.txt", "data\pdk_web-historian_2.txt"]
dataFramePassiveDataKit = pandas.DataFrame()
for dataFile in dataFilesPassiveDataKit:
    dataFramePassiveDataKit = dataFramePassiveDataKit.append(pandas.read_csv(dataFile, sep="\t"))

The _Source_ column contains the _Web Historian_ ID of the participants, which allows us to link this data with the Qualtrics data. Let's set the _Source_ as the index.

In [ ]:
dataFramePassiveDataKit = dataFramePassiveDataKit.set_index("Source")
dataFramePassiveDataKit.head(3)

If all went well, we should have browsing history data for each of the Qualtrics participants. Let's verify this by examining the intersection and the difference between the index of the Qualtrics dataframe and that of the Passive Data Kit dataframe.

In [ ]:
participantsPassiveDataKit = dataFramePassiveDataKit.index.unique()
len(participantsPassiveDataKit)

To find the participants who are found in both dataframes, let's take the intersection. Note that we are not using the Qualtrics index, which are the hashed email addresses, but the list of Web Historian ID's which we had previously placed into a list.

In [ ]:
participantsBothDataFrames = participantsPassiveDataKit.intersection(qualtricsIdListUnionWaves)
len(participantsBothDataFrames)

It might be that some ID's can be found in the Passive Data Kit dataframe which are not found in the Qualtrics dataframe.

In [ ]:
participantsPassiveDataKit.difference(qualtricsIdListUnionWaves)

Vice versa, some ID's are possibly found in the Qualtrics dataframe which are not found in the Passive Data Kit dataframe.

In [ ]:
pandas.Index(qualtricsIdListUnionWaves).difference(participantsPassiveDataKit)

For simplicity, let's just remove all these outliers and construct smaller dataframes for each wave. First, we adjust the list of participants.

In [ ]:
participantsBothDataFramesFirstWave = participantsBothDataFrames.intersection(qualtricsIdListFirstWave)
participantsBothDataFramesSecondWave = participantsBothDataFrames.intersection(qualtricsIdListSecondWave)
participantsBothDataFramesUnionWaves = participantsBothDataFrames.intersection(qualtricsIdListUnionWaves)

We now filter the Passive Data Kit dataframe on the relevant participants.

In [ ]:
passiveDataKitFirstWave = dataFramePassiveDataKit.loc[participantsBothDataFramesFirstWave]
passiveDataKitFirstWave.head(3)

We now filter the Passive Data Kit dataframe on the relevant participants.

In [ ]:
passiveDataKitSecondWave = dataFramePassiveDataKit.loc[participantsBothDataFramesSecondWave]
passiveDataKitSecondWave.head(3)

We now filter the Passive Data Kit dataframe on the relevant participants. Note that this dataframe may contain duplicates from the browsing history of people who entered both waves.

In [ ]:
passiveDataKitUnionWaves = dataFramePassiveDataKit.loc[participantsBothDataFramesUnionWaves]
passiveDataKitUnionWaves.head(3)

### Simple Metrics

It is interesting to see how long participants keep their browsing history for. Let's construct a column which measures the time between the first URL entry and the time the data was uploaded.

In [ ]:
passiveDataKitFirstWave['TimeSinceUrlVisit'] = passiveDataKitFirstWave['Recorded Timestamp'] - passiveDataKitFirstWave['Timestamp']

The maximum of this time difference indicates the length of time since the browsing history was last cleared.

In [ ]:
isMaxTimeSinceUrlVisitPassiveDataKit = passiveDataKitFirstWave.groupby(['Source'])['TimeSinceUrlVisit'].transform(max) == passiveDataKitFirstWave['TimeSinceUrlVisit']

This time difference can be placed into a new dataframe.

In [ ]:
browsingHistoryTimeRange = passiveDataKitFirstWave[isMaxTimeSinceUrlVisitPassiveDataKit]['TimeSinceUrlVisit']

We can plot the browsing history time range as a histogram.

In [ ]:
(browsingHistoryTimeRange/60/60/24).plot(kind='hist',title='Browsing history time range').get_figure().savefig("figures/BrowsingHistoryTimeRange.png")

We can also count the number of URL's per person.

In [ ]:
countUrlsPerPersonPassiveDataKit = passiveDataKitUnionWaves.groupby(['Source'])['URL'].count()

And again plot this in a histogram.

In [ ]:
countUrlsPerPersonPassiveDataKit.plot(kind='hist',title='URL\'s per person').get_figure().savefig("figures/CountUrlsPerPerson.png")

For the data entries which belong to the first wave, let's find the timestamp of their last entry.

In [ ]:
isEndOfFirstWave = passiveDataKitFirstWave.groupby(['Source'])['Timestamp'].transform(max) == passiveDataKitFirstWave['Timestamp']

In [ ]:
dataFramePassiveDataKitEndOfFirstWave = passiveDataKitFirstWave[isEndOfFirstWave][['Timestamp']]
dataFramePassiveDataKitEndOfFirstWave.head(3)

For participants who entered both waves, let's change their Web Historian ID to the ID of the second wave.

In [ ]:
dataFramePassiveDataKitEndOfFirstWave.index = dataFramePassiveDataKitEndOfFirstWave.index.map(qualtricsIdDictionary)
dataFramePassiveDataKitEndOfFirstWave = dataFramePassiveDataKitEndOfFirstWave.loc[participantsBothDataFramesSecondWave]
dataFramePassiveDataKitEndOfFirstWave = dataFramePassiveDataKitEndOfFirstWave.rename(columns={"Timestamp": "TimestampEndOfFirstWave"})
dataFramePassiveDataKitEndOfFirstWave

Now we can append this column to the Passive Data Kit dataframe.

In [ ]:
dataFramePassiveDataKitSecondWave = pandas.concat([passiveDataKitSecondWave, dataFramePassiveDataKitEndOfFirstWave], axis=1, join_axes=[passiveDataKitSecondWave.index])
dataFramePassiveDataKitSecondWave.head(3)

This allows us to find all URL entries related to the period between the two waves.

In [ ]:
isBetweenWaves = dataFramePassiveDataKitSecondWave["TimestampEndOfFirstWave"] < dataFramePassiveDataKitSecondWave["Timestamp"]
dataFramePassiveDataKitBetweenWaves = dataFramePassiveDataKitSecondWave[isBetweenWaves]
dataFramePassiveDataKitBetweenWaves.head(3)

We can count the number of URL's per person.

In [ ]:
countUrlsPassiveDataKitBetweenWaves = dataFramePassiveDataKitBetweenWaves.groupby(['Source'])['URL'].count()

And plot this in a histogram.

In [ ]:
countUrlsPassiveDataKitBetweenWaves.plot(kind='hist',title='URL\'s per person between waves').get_figure().savefig("figures/CountUrlsBetweenWaves.png")

We can add a bit more data to these URL counts. Let's start by finding the timestamp of the end of the second wave for each participant.

In [ ]:
isMaxTimeStampSecondWavePassiveDataKit = dataFramePassiveDataKitBetweenWaves.groupby(['Source'])['Timestamp'].transform(max) == dataFramePassiveDataKitBetweenWaves['Timestamp']
dataFramePassiveDataKitEndOfSecondWave = dataFramePassiveDataKitBetweenWaves[isMaxTimeStampSecondWavePassiveDataKit][['Timestamp']]
dataFramePassiveDataKitEndOfSecondWave = dataFramePassiveDataKitEndOfSecondWave.rename(columns={"Timestamp": "TimestampEndOfSecondWave"})
dataFramePassiveDataKitEndOfSecondWave.head(3)

Now we can join this with the timestamp of the end of the first wave and with the URL counts, and we can determine some additional metrics.

In [ ]:
countUrlsAndTimeBetweenWaves = pandas.concat([dataFramePassiveDataKitEndOfFirstWave, dataFramePassiveDataKitEndOfSecondWave, countUrlsPassiveDataKitBetweenWaves], axis=1, join_axes=[dataFramePassiveDataKitEndOfFirstWave.index])
countUrlsAndTimeBetweenWaves["TimeDifference"] = countUrlsAndTimeBetweenWaves["TimestampEndOfSecondWave"] - countUrlsAndTimeBetweenWaves["TimestampEndOfFirstWave"]
countUrlsAndTimeBetweenWaves["UrlsPerDay"] = countUrlsAndTimeBetweenWaves["URL"] / countUrlsAndTimeBetweenWaves["TimeDifference"] * 60 * 60 * 24
countUrlsAndTimeBetweenWaves = countUrlsAndTimeBetweenWaves.rename(columns={"URL": "UrlCount"})
countUrlsAndTimeBetweenWaves.to_csv("tables\CountUrlsAndTimeBetweenWaves.csv")
countUrlsAndTimeBetweenWaves.head(3)

For example, the URL count per day can also be plotted in a histogram.

In [ ]:
countUrlsAndTimeBetweenWaves["UrlsPerDay"].plot(kind='hist',title='URL\'s per person per day').get_figure().savefig("figures/CountUrlsPerDayBetweenWaves.png")

Let's now see if we can determine whether a URL was visited on a mobile device by checking for a mobile subdomain.

In [ ]:
isMobileUrl = dataFramePassiveDataKitBetweenWaves['URL'].str.match("^(http:\/\/|https:\/\/)?(m|mob|mobile)\.([a-z0-9]+)*\.[a-z]{2,5}(:[0-9]{1,5})?(\/.*)?$")

We can count mobile URL's per participant.

In [ ]:
countMobileUrlsBetweenWaves = dataFramePassiveDataKitBetweenWaves[isMobileUrl].groupby(['Source'])['URL'].count()

It would also be interesting to see if the mobile URL count depends on whether or not the participants were asked to sync their mobile devices before uploading their browsing history. This can be done by joining data from the Qualtrics survey.

In [ ]:
countMobileUrlsBetweenWaves = pandas.concat([dataFrameQualtrics.loc[participantsIntersectionWaves].set_index("idRcvd_2")[["sync_1"]], countMobileUrlsBetweenWaves], axis=1, sort=False)
countMobileUrlsBetweenWaves
countMobileUrlsBetweenWaves.to_csv("tables\CountMobileUrlsBetweenWaves.csv")
countMobileUrlsBetweenWaves.head(3)

Let's combine all participants into two groups, depending on whether or not they synced their borwsing history.

In [ ]:
countMobileUrlsBetweenWaves.groupby(["sync_1"])["URL"].sum().to_frame().rename(columns={"URL": "CountMobileUrls"})

Let's count the number of visit to the Dutch newswebsite _NOS.nl_ by first defining a regular expression.

In [ ]:
isNOSUrl = dataFramePassiveDataKitBetweenWaves['URL'].str.match("^(http:\/\/|https:\/\/)?([a-z0-9]+\.)?nos.nl(\/.*)?$")

We can count the URL's per participant.

In [ ]:
countNOSUrlsBetweenWaves = dataFramePassiveDataKitBetweenWaves[isNOSUrl].groupby(['Source'])['URL'].count().to_frame()
countNOSUrlsBetweenWaves = countNOSUrlsBetweenWaves.rename(columns={"URL": "Count nos.nl"})
countNOSUrlsBetweenWaves.to_csv("tables\CountNOSBetweenWaves.csv")
countNOSUrlsBetweenWaves.head(3)

Let's count the number of visit to the UK news website _The Guardian_ by first defining a regular expression.

In [ ]:
isGuardianUrl = dataFramePassiveDataKitBetweenWaves['URL'].str.match("^(http:\/\/|https:\/\/)?([a-z0-9]+\.)?theguardian.com(\/.*)?$")

We can count the URL's per participant.

In [ ]:
countGuardianUrlsBetweenWaves = dataFramePassiveDataKitBetweenWaves[isGuardianUrl].groupby(['Source'])['URL'].count().to_frame()
countGuardianUrlsBetweenWaves = countGuardianUrlsBetweenWaves.rename(columns={"URL": "Count The Guardian"})
countGuardianUrlsBetweenWaves.to_csv("tables\CountTheGuardianBetweenWaves.csv")
countGuardianUrlsBetweenWaves.head(3)

Let's count the number of visit to _Facebook_ by first defining a regular expression.

In [ ]:
isFacebookUrl = dataFramePassiveDataKitBetweenWaves['URL'].str.match("^(http:\/\/|https:\/\/)?([a-z0-9]+\.)?facebook.com(\/.*)?$")

We can count the URL's per participant.

In [ ]:
countFacebookUrlsBetweenWaves = dataFramePassiveDataKitBetweenWaves[isFacebookUrl].groupby(['Source'])['URL'].count().to_frame()
countFacebookUrlsBetweenWaves = countFacebookUrlsBetweenWaves.rename(columns={"URL": "Count Facebook"})
countFacebookUrlsBetweenWaves.to_csv("tables\CountFacebookBetweenWaves.csv")
countFacebookUrlsBetweenWaves.head(3)

Let's count the number of visit a selection of news websites.

In [ ]:
newsList = 'volkskrant|telegraaf|nu\.|ad\.|nos\.nl|fd\.|rd\.|nd\.nl'\
'|metro\.|nieuws|dagblad|bd\.|krant|nrc|geenstijl|parool|trouw\.|fok\.'\
'|destentor|gelderlander|ed\.nl|bndestem|pzc\.nl|tubantia|limburger|'\
'zwartewaterkrant|gva|demorgen|tijd\.be|hbvl|hln|standaard\.be'\
'|dvhn|lc\.|gooieneemlander|barneveldsekrant|amsterdamweekly|courant|'\
'herautonline|mooirooi|gezinsbode'

This list is added to a regular expression.

In [ ]:
isNewsUrl = dataFramePassiveDataKitBetweenWaves['URL'].str.match("^(http:\/\/|https:\/\/)?([a-z0-9]+\.)?(" + newsList + ")(.*)?$")

We can count the URL's per participant.

In [ ]:
countNewsUrlsBetweenWaves = dataFramePassiveDataKitBetweenWaves[isNewsUrl].groupby(['Source'])['URL'].count().to_frame()
countNewsUrlsBetweenWaves = countNewsUrlsBetweenWaves.rename(columns={"URL": "Count News"})
countNewsUrlsBetweenWaves.to_csv("tables\CountNewsBetweenWaves.csv")
countNewsUrlsBetweenWaves.head(3)

Let's also do this for the first wave of data, which contains more URL in total. The computation may be slow.

In [ ]:
isNewsUrlFirstWave = passiveDataKitFirstWave['URL'].str.match("^(http:\/\/|https:\/\/)?([a-z0-9]+\.)?(" + newsList + ")(.*)?$")

In [ ]:
countNewsUrlsFirstWave = passiveDataKitFirstWave[isNewsUrlFirstWave].groupby(['Source'])['URL'].count().to_frame()
countNewsUrlsFirstWave = countNewsUrlsFirstWave.rename(columns={"URL": "Count News"})
countNewsUrlsFirstWave.to_csv("tables\CountNewsFirstWave.csv")
countNewsUrlsFirstWave.head(3)

End of _Jupyter Notebook_.